In [ ]:
# Entrena un modelo de Gradient Boosted Trees
# para predecir el consumo eléctrico
# y la cantidad de usuarios.

# Tomado y modificado a partir de:
# https://www.datacamp.com/community/tutorials/xgboost-in-python
# https://towardsdatascience.com/a-beginners-guide-to-xgboost-87f5d4c30ed7

import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import ( mean_squared_error,
    precision_score, recall_score, accuracy_score )
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
path_drive = "/content/drive/MyDrive/Colab/" 
path_data = path_drive + "data/"
fname = "data.csv"

data = pd.read_csv(path_data + fname)

# Información utilizada para el modelo.
data.head()

In [ ]:
# Medias.
data.iloc[:,2:].mean(axis = 0)

In [ ]:
# Desviaciones estándar.
data.iloc[:,2:].std(axis = 0)

In [ ]:
# Cuartiles.
data.iloc[:,2:].quantile([0.25, 0.5, 0.75], axis = 0)

In [ ]:
# Matriz de correlación de Pearson.
data.iloc[:,2:].corr()

In [ ]:
# Escogemos el conjunto de features y de variables a predecir.
# Escogemos solo uno de los conjuntos a predecir.
i = 1
X, Y = data.iloc[:,6:], data.iloc[:,i + 1]

# Separamos en conjuntos de entrenamiento y de prueba.
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size = 0.2, random_state = 123 )

# Features a utilizar.
X.head()

In [ ]:
# Variable a predecir.
Y.head()

In [ ]:
# Método 0.
# Regresión lineal.

# Creamos el regresor.
lin_reg = LinearRegression()

# Entrenamos el modelo.
lin_reg.fit(X_train, Y_train)

# Evaluamos el modelo.
preds = lin_reg.predict(X_test)

# Calculamos el error.
rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse}")

# Escoge las mejores predicciones ??????
# ¿¿Por qué??
best_preds = np.asarray([np.argmax(line) for line in preds])

# Más medidas para evaluar el modelo.
precision = precision_score(Y_test, best_preds, average='macro')
recall = recall_score(Y_test, best_preds, average='macro')
accuracy = accuracy_score(Y_test, best_preds)
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"Accuracy = {accuracy}")

In [ ]:
# Método 1.
# Entrenamiento simple.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 50,
    #"min_child_weight" : 5,
    "alpha": 10,
    #"gamma": 0.2,
    "n_estimators": 100,
    "tree_method": "gpu_hist"
    }

# Creamos el regresor con los hiperparámetros.
xg_reg = xgb.XGBRegressor( **params )

# Entrenamos el modelo.
xg_reg.fit(X_train, Y_train, verbose = True)

# Evaluamos el modelo.
preds = xg_reg.predict(X_test)

# Calculamos el error.
rmse = np.sqrt(mean_squared_error(Y_test, preds))
print(f"RMSE: {rmse}")

# Escoge las mejores predicciones ??????
# ¿¿Por qué??
best_preds = np.asarray([np.argmax(line) for line in preds])

# Más medidas para evaluar el modelo.
precision = precision_score(Y_test, best_preds, average='macro')
recall = recall_score(Y_test, best_preds, average='macro')
accuracy = accuracy_score(Y_test, best_preds)
print(f"Precision = {precision}")
print(f"Recall = {recall}")
print(f"Accuracy = {accuracy}")

# Otras formas de entrenar el modelo

Primero hacer funcionar la principal estrategia.

In [ ]:
# Método 2.
# Ajuste de hiperparámetros.

# Hiperparámetros.
params = {
    "colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7 ],
    "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [10, 20, 30, 40, 50, 60, 70],
    #"min_child_weight" : [ 1, 3, 5, 7 ],
    "alpha": [7, 8, 9, 10, 11, 12, 13],
    #"gamma": [0.0, 0.1, 0.2 , 0.3, 0.4],
    "n_estimators": [50, 100, 200, 300, 400, 500, 700],
    }

# Creamos el regresor.
xg_reg = xgb.XGBRegressor(
    objective = "reg:squarederror",
    tree_method = "gpu_hist" )

# Ajustador de hiperparámetros con Cross Validation.
grid = GridSearchCV(xg_reg, params,
    n_jobs = -1, cv = 5, verbose = 2
    scoring = "neg_mean_absolute_error")

# Entrenamos el modelo.
grid.fit(X_train, Y_train)

# Evaluamos el modelo.
preds = grid.predict(X_test)

# Calculamos el error.
rmse = np.sqrt(mean_squared_error(Y_test, preds))
print("RMSE: %f" % (rmse))

# Escoge las mejores predicciones ??????
# ¿¿Por qué??
best_preds = np.asarray([np.argmax(line) for line in preds])

# Más medidas para evaluar el modelo.
print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

In [ ]:
# Método 3.
# Cross Validation con XGBoost.

# Hiperparámetros.
params = {
    "objective": "reg:squarederror",
    "colsample_bytree": 0.3,
    "learning_rate": 0.1,
    "max_depth": 10,
    #"min_child_weight" : 5,
    "alpha": 10,
    #"gamma": 0.2,
    "n_estimators": 100,
    "tree_method": "gpu_hist"
    }

# Ajustamos el DataFrame al formato de XGBoost.
data_dmatrix = xgb.DMatrix(data = X, label = Y)

# Realizamos el entrenamiento con Cross Validation.
cv_results = xgb.cv(dtrain = data_dmatrix, params = params, nfold = 3,
    num_boost_round = 50, early_stopping_rounds = 10, metrics = "rmse",
    as_pandas = True, seed = 123)

print((cv_results["test-rmse-mean"]).tail(1))